In [1]:
import pandas as pd
from sklearn import svm
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers import Dense

2023-04-25 12:22:37.004450: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-25 12:22:37.051776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 12:22:38.254312: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv("german_encoded.csv")

In [3]:
X_1 = data.iloc[:,:8]
X_2 = data.iloc[:,9:]
X = pd.concat([X_1, X_2], axis=1)
y = data.iloc[:,8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# SVM

In [6]:
def linear():
    model = LinearSVC()
    scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1_macro', error_score="raise")
    print('CV score:', scores.mean(), "+/-", scores.std())
    
linear()

CV score: 0.30773972500561986 +/- 0.02215816082864043


Results:
- CV score: 0.30773972500561986 +/- 0.02215816082864043

In [7]:
model = SVC()
scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1_macro', error_score="raise")
print('CV score:', scores.mean(), "+/-", scores.std())

CV score: 0.31311853560001346 +/- 0.026548623659016182


Results:
- CV score: 0.31311853560001346 +/- 0.026548623659016182

## GridSearch

In [8]:
param_grid= {'kernel': ('linear', 'rbf'),'C': [5,10, 20, 30, 40, 100], 'gamma': [0.05, 0.01, 0.005, 0.001]}
base_estimator = SVC()
sh = GridSearchCV(base_estimator, param_grid, cv=10,scoring = 'f1_macro').fit(X_train, y_train)

In [9]:
param = sh.best_params_
score = sh.score(X_train, y_train)
print("Best param:", param)
print("Best score:",score)

Best param: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
Best score: 0.8471626227980072


Results:
- {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}, score = 0.8471626227980072

# SGD

In [10]:
model = SGDClassifier(loss = 'log_loss')

scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1_macro', error_score="raise")
print(scores.mean())

0.33776965686804045


Results:
- 0.33776965686804045

In [11]:
param_grid = {
    'alpha': [0.0001, 0.00005, 0.00007, 0.00009],
    'penalty': ['l2','l1'],
    'eta0': [0.0001, 0.001, 0.005, 0.007, 0.009, 0.01, 0.015]
}

# create a GridSearchCV object with the specified hyperparameter grid
grids = GridSearchCV(model, param_grid, cv=10, scoring = 'f1_macro', verbose = 1)

# fit the grid search object to the data
grids.fit(X_train , y_train)

# print the best hyperparameters and the corresponding score
print("Best hyperparameters: ", grids.best_params_)
print("Best score: ", grids.best_score_)

Fitting 10 folds for each of 56 candidates, totalling 560 fits
Best hyperparameters:  {'alpha': 0.0001, 'eta0': 0.009, 'penalty': 'l1'}
Best score:  0.37432158419405825


Results:
- {'alpha': 0.0001, 'eta0': 0.009, 'penalty': 'l1'}, score = 0.37432158419405825

# RandomForest

In [12]:
model = RandomForestClassifier()
scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1_macro', error_score="raise")
print('CV score:', scores.mean(), "+/-", scores.std())

CV score: 0.3364518878638053 +/- 0.024641888167460192


Results:
- CV score: 0.3364518878638053 +/- 0.024641888167460192

In [13]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [15]:
from sklearn.tree import export_text
# Print out the trees in the ensemble
for i, tree in enumerate(model.estimators_):
    print(f"Tree {i}:\n{export_text(tree)}\n")

Tree 0:
|--- feature_14 <= 1.50
|   |--- feature_1 <= 1.50
|   |   |--- feature_11 <= 0.50
|   |   |   |--- feature_0 <= 2.50
|   |   |   |   |--- feature_19 <= 1.50
|   |   |   |   |   |--- feature_17 <= 0.50
|   |   |   |   |   |   |--- feature_0 <= 1.50
|   |   |   |   |   |   |   |--- feature_10 <= 0.50
|   |   |   |   |   |   |   |   |--- feature_3 <= 1.50
|   |   |   |   |   |   |   |   |   |--- feature_9 <= 1.50
|   |   |   |   |   |   |   |   |   |   |--- class: 3.0
|   |   |   |   |   |   |   |   |   |--- feature_9 >  1.50
|   |   |   |   |   |   |   |   |   |   |--- feature_5 <= 2.00
|   |   |   |   |   |   |   |   |   |   |   |--- class: 3.0
|   |   |   |   |   |   |   |   |   |   |--- feature_5 >  2.00
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |--- feature_3 >  1.50
|   |   |   |   |   |   |   |   |   |--- feature_5 <= 1.00
|   |   |   |   |   |   |   |   |   |   |--- feature_0 <= 0.50
|   |   |   |   |   |   |   |   |   |  

In [16]:
# Get the list of decision tree estimators
estimators = model.estimators_

# Find the best decision tree
best_tree = None
best_score = -1

for estimator in estimators:
    score = estimator.score(X_train, y_train)
    if score > best_score:
        best_tree = estimator
        best_score = score

# Print the best tree
tree_str = export_text(best_tree)
print(tree_str)

|--- feature_16 <= 1.50
|   |--- feature_6 <= 2.50
|   |   |--- feature_0 <= 0.50
|   |   |   |--- feature_13 <= 1.50
|   |   |   |   |--- feature_13 <= 0.50
|   |   |   |   |   |--- feature_3 <= 8.00
|   |   |   |   |   |   |--- feature_2 <= 0.50
|   |   |   |   |   |   |   |--- class: 2.0
|   |   |   |   |   |   |--- feature_2 >  0.50
|   |   |   |   |   |   |   |--- feature_8 <= 1.50
|   |   |   |   |   |   |   |   |--- feature_11 <= 2.00
|   |   |   |   |   |   |   |   |   |--- feature_1 <= 2.50
|   |   |   |   |   |   |   |   |   |   |--- feature_7 <= 2.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 4
|   |   |   |   |   |   |   |   |   |   |--- feature_7 >  2.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |   |--- feature_1 >  2.50
|   |   |   |   |   |   |   |   |   |   |--- feature_4 <= 2.00
|   |   |   |   |   |   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |   |   |   |   |   |   |--- featu

/opt/pub/eb/apps/all/DavidsonJupyter/2021-common-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/opt/pub/eb/apps/all/DavidsonJupyter/2021-common-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/opt/pub/eb/apps/all/DavidsonJupyter/2021-common-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/opt/pub/eb/apps/all/DavidsonJupyter/2021-common-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/opt/pub/eb/apps/all/DavidsonJupyter/2021-common-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/base

# k-NN

In [17]:
param_grid= {'n_neighbors': [15,20,25],'p': [1,2]}
models = KNeighborsClassifier()
ch = GridSearchCV(models, param_grid, cv=10,scoring = 'f1_micro').fit(X_train, y_train)

In [18]:
k_param = ch.best_params_
k_score = ch.best_score_
print('best param is', k_param)
print('best score is', k_score)

best param is {'n_neighbors': 25, 'p': 1}
best score is 0.6200000000000001


Results:
- {'n_neighbors': 25, 'p': 1}, score = 0.6200000000000001

# Decision Tree

In [19]:
# Define the decision tree classifier and the hyperparameters to optimize
clf = DecisionTreeClassifier()
param_grid = {'max_depth': [7, 8, 9],
              'min_samples_split': [9, 10, 11],
              'min_samples_leaf': [1, 2, 3, 4, 5]}

# Perform the grid search with cross-validation
dtgs = GridSearchCV(clf, param_grid=param_grid, cv=10,scoring = 'f1_micro')
dtgs.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding evaluation metric
print("Best hyperparameters:", dtgs.best_params_)
print("Best score:", dtgs.best_score_)

Best hyperparameters: {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 11}
Best score: 0.5733333333333334


Results:
- {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 11}, score = 0.5733333333333334

# Neural Network

In [20]:
# build the neural network model
nn = Sequential()
nn.add(Dense(10, input_dim=20, activation='relu'))
nn.add(Dense(5, activation='relu'))
nn.add(Dense(1, activation='sigmoid'))

# compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the model to the training data
nn.fit(X_train, y_train, epochs=50, batch_size=32)

# evaluate the model on the test data
loss, accuracy = nn.evaluate(X_train, y_train)

# print the test accuracy
print('Test accuracy:', accuracy)

Epoch 1/50


2023-04-25 12:33:13.495362: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-25 12:33:13.495403: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: gpu0.ada.davidson.edu
2023-04-25 12:33:13.495409: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: gpu0.ada.davidson.edu
2023-04-25 12:33:13.495511: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 530.30.2
2023-04-25 12:33:13.495526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 530.30.2
2023-04-25 12:33:13.495530: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 530.30.2


24/24 [==============================] - 1s 2ms/step - loss: 1.0392 - accuracy: 0.2027
Epoch 2/50
24/24 [==============================] - 0s 1ms/step - loss: -0.6038 - accuracy: 0.2907
Epoch 3/50
24/24 [==============================] - 0s 1ms/step - loss: -2.2788 - accuracy: 0.3107
Epoch 4/50
24/24 [==============================] - 0s 1ms/step - loss: -5.0401 - accuracy: 0.3107
Epoch 5/50
24/24 [==============================] - 0s 1ms/step - loss: -9.2369 - accuracy: 0.3107
Epoch 6/50
24/24 [==============================] - 0s 1ms/step - loss: -15.0069 - accuracy: 0.3107
Epoch 7/50
24/24 [==============================] - 0s 1ms/step - loss: -22.6991 - accuracy: 0.3107
Epoch 8/50
24/24 [==============================] - 0s 1ms/step - loss: -32.2110 - accuracy: 0.3107
Epoch 9/50
24/24 [==============================] - 0s 1ms/step - loss: -44.3026 - accuracy: 0.3107
Epoch 10/50
24/24 [==============================] - 0s 1ms/step - loss: -59.5848 - accuracy: 0.3107
Epoch 11/50
24/2

Results:
- Test accuracy: 0.31066668033599854, 50 epochs, 10 in second layer, 5 in third layer, batch size = 32

# Scores

SVM: {'C': 20, 'gamma': 0.01, 'kernel': 'rbf'}, score = 0.8290713574757445

SGD: {'alpha': 7e-05, 'eta0': 0.005, 'penalty': 'l2'}, score = 0.6616935571076444

RandomForest: CV score: 0.663357274635801 +/- 0.04059552799388036

k-NN: {'n_neighbors': 20, 'p': 1}, score = 0.752

Decision Tree: {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 10}, score = 0.72

Neural Network: Test accuracy: 0.699999988079071, 50 epochs, 10 in second layer, batch size = 32

**SVM is the highest**